In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from mats.loading import load_all, load_sae

model, data, sae_l1 = load_all(model_name="gelu-2l", run_id="l1")
sae_l0 = load_sae(run_id="l0")

Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cuda
Changing model dtype to torch.float16
Model device: cuda:0
Tokens shape: torch.Size([215402, 128]), dtype: torch.int64, device: cuda:0
Loading l1 from HuggingFace at gelu-2l_L1_16384_mlp_out_50
{'act_name': 'blocks.1.hook_mlp_out',
 'act_size': 512,
 'batch_size': 4096,
 'beta1': 0.9,
 'beta2': 0.99,
 'buffer_batches': 12288,
 'buffer_mult': 384,
 'buffer_size': 1572864,
 'd_mlp': 512,
 'device': 'cuda:0',
 'dict_mult': 32,
 'dict_size': 16384,
 'enc_dtype': 'fp32',
 'l1_coeff': 0.0003,
 'layer': 1,
 'lr': 0.0001,
 'model_batch_size': 512,
 'model_name': 'gelu-2l',
 'num_tokens': 2000000000,
 'remove_rare_dir': False,
 'seed': 50,
 'seq_len': 128,
 'site': 'mlp_out'}
Encoder device: cuda:0
Loading l0 from HuggingFace at gelu-2l_L0_16384_mlp_out_51


gelu-2l_L0_16384_mlp_out_51_cfg.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

{'act_name': 'blocks.0.hook_mlp_out',
 'act_size': 512,
 'batch_size': 4096,
 'beta1': 0.9,
 'beta2': 0.99,
 'buffer_batches': 12288,
 'buffer_mult': 384,
 'buffer_size': 1572864,
 'd_mlp': 512,
 'device': 'cuda:1',
 'dict_mult': 32,
 'dict_size': 16384,
 'enc_dtype': 'fp32',
 'l1_coeff': 0.0003,
 'layer': 0,
 'lr': 0.0001,
 'model_batch_size': 512,
 'model_name': 'gelu-2l',
 'num_tokens': 2000000000,
 'remove_rare_dir': False,
 'seed': 51,
 'seq_len': 128,
 'site': 'mlp_out'}


gelu-2l_L0_16384_mlp_out_51.pt:   0%|          | 0.00/67.2M [00:00<?, ?B/s]

Encoder device: cuda:0


In [5]:
from transformer_lens import utils

utils.get_act_name("dsfasdf", 1),

('blocks.1.hook_dsfasdf',)

In [6]:
# Check that it's two layers
len(model.blocks)

2

In [9]:
# Shapes - gelu-2l

print(model.blocks[0].mlp.W_in.shape)
print(sae_l1.W_enc.shape)

torch.Size([512, 2048])
torch.Size([512, 16384])


In [10]:
# That doesn't look right... let's check this for the old model

model, data, sae = load_all()

print(model.blocks[0].mlp.W_in.shape)
print(sae.W_enc.shape)

Loaded pretrained model gelu-1l into HookedTransformer
Moving model to device:  cuda
Changing model dtype to torch.float16
Model device: cuda:0
Tokens shape: torch.Size([215402, 128]), dtype: torch.int64, device: cuda:0
Loading run1 from HuggingFace at 25
{'batch_size': 4096,
 'beta1': 0.9,
 'beta2': 0.99,
 'buffer_batches': 12288,
 'buffer_mult': 384,
 'buffer_size': 1572864,
 'd_mlp': 2048,
 'dict_mult': 8,
 'enc_dtype': 'fp32',
 'l1_coeff': 0.0003,
 'lr': 0.0001,
 'model_batch_size': 512,
 'num_tokens': 2000000000,
 'seed': 52,
 'seq_len': 128}
Encoder device: cuda:0
torch.Size([512, 2048])
torch.Size([2048, 16384])


In [15]:
# Verify linearization still works at L0

from mats.linearization import analyze_linearized_feature

analyze_linearized_feature(
    feature_idx=100,
    sample_idx=30,
    token_idx=30,
    model=model,
    data=data,
    encoder=sae_l0,
    layer=0,
    # batch_size=32,
    n_tokens=10,
    mlp_out=True,
)

RuntimeError: size mismatch, got input (1), mat (1x512), vec (2048)

In [16]:
# Repeat at L1 (this multiplies by the wrong features, but I'm just seeing what breaks)

analyze_linearized_feature(
    feature_idx=100,
    sample_idx=30,
    token_idx=30,
    model=model,
    layer=1,
    data=data,
    encoder=sae_l1,
    batch_size=32,
    n_tokens=10
)

TypeError: analyze_linearized_feature() got an unexpected keyword argument 'batch_size'

In [ ]:
load_all(model_name="gelu-2l", run="l1")

Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cuda
Changing model dtype to torch.float16
Model device: cuda:0
Tokens shape: torch.Size([215402, 128]), dtype: torch.int64, device: cuda:0
{'batch_size': 4096,
 'beta1': 0.9,
 'beta2': 0.99,
 'buffer_batches': 12288,
 'buffer_mult': 384,
 'buffer_size': 1572864,
 'd_mlp': 2048,
 'dict_mult': 8,
 'enc_dtype': 'fp32',
 'l1_coeff': 0.0003,
 'lr': 0.0001,
 'model_batch_size': 512,
 'num_tokens': 2000000000,
 'seed': 52,
 'seq_len': 128}
Encoder device: cuda:0


(HookedTransformer(
   (embed): Embed()
   (hook_embed): HookPoint()
   (pos_embed): PosEmbed()
   (hook_pos_embed): HookPoint()
   (blocks): ModuleList(
     (0-1): 2 x TransformerBlock(
       (ln1): LayerNormPre(
         (hook_scale): HookPoint()
         (hook_normalized): HookPoint()
       )
       (ln2): LayerNormPre(
         (hook_scale): HookPoint()
         (hook_normalized): HookPoint()
       )
       (attn): Attention(
         (hook_k): HookPoint()
         (hook_q): HookPoint()
         (hook_v): HookPoint()
         (hook_z): HookPoint()
         (hook_attn_scores): HookPoint()
         (hook_pattern): HookPoint()
         (hook_result): HookPoint()
       )
       (mlp): MLP(
         (hook_pre): HookPoint()
         (hook_post): HookPoint()
       )
       (hook_attn_in): HookPoint()
       (hook_q_input): HookPoint()
       (hook_k_input): HookPoint()
       (hook_v_input): HookPoint()
       (hook_mlp_in): HookPoint()
       (hook_attn_out): HookPoint()
       (ho